In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Set a seed for reproducibility
np.random.seed(42)

# Generate random time series data
date_rng = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
data = np.random.randn(len(date_rng))
ts_df = pd.DataFrame(data, columns=['value'], index=date_rng)

# Split the data into training and testing sets
train_size = int(len(ts_df) * 0.8)
train, test = ts_df[0:train_size], ts_df[train_size:]

# Function to evaluate the model and print RMSE
def evaluate_model(model, test, pred):
    mse = mean_squared_error(test, pred)
    rmse = np.sqrt(mse)
    print(f'Model: {model}, RMSE: {rmse}')

# SARIMA model
sarima_model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
sarima_fit = sarima_model.fit(disp=False)
sarima_pred = sarima_fit.get_forecast(steps=len(test)).predicted_mean
evaluate_model('SARIMA', test, sarima_pred)

# Exponential Smoothing (ETS) model
ets_model = ExponentialSmoothing(train, seasonal='add', seasonal_periods=7)
ets_fit = ets_model.fit()
ets_pred = ets_fit.predict(start=test.index[0], end=test.index[-1])
evaluate_model('ETS', test, ets_pred)

# Prophet model
prophet_model = Prophet()
prophet_train = pd.DataFrame({'ds': train.index, 'y': train['value']})
prophet_model.fit(prophet_train)
future = pd.DataFrame({'ds': test.index})
prophet_pred = prophet_model.predict(future)['yhat'].values
evaluate_model('Prophet', test, prophet_pred)

# Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(np.arange(len(train)).reshape(-1, 1), train['value'])
rf_pred = rf_model.predict(np.arange(len(test)).reshape(-1, 1))
evaluate_model('Random Forest', test, rf_pred)

# XGBoost model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(np.arange(len(train)).reshape(-1, 1), train['value'])
xgb_pred = xgb_model.predict(np.arange(len(test)).reshape(-1, 1))
evaluate_model('XGBoost', test, xgb_pred)

# LightGBM model
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model.fit(np.arange(len(train)).reshape(-1, 1), train['value'])
lgbm_pred = lgbm_model.predict(np.arange(len(test)).reshape(-1, 1))
evaluate_model('LightGBM', test, lgbm_pred)

# LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

lstm_model = create_lstm_model()
lstm_train = train['value'].values.reshape(-1, 1, 1)
lstm_model.fit(lstm_train, train['value'], epochs=50, verbose=0)
lstm_pred = lstm_model.predict(np.arange(len(test)).reshape(-1, 1, 1)).flatten()
evaluate_model('LSTM', test, lstm_pred)


ModuleNotFoundError: No module named 'keras'